# Compressing neural networks with Gaussian mixture priors

---
## Introduction
In this project we have implemented the paper titled [Soft weight-sharing for Neural Network compression](https://arxiv.org/abs/1702.04008) by Ullrich, Meeds and Welling. The main idea of the paper is to introduce a prior on the weights of a pre-trained network that encourages a lot of weights to go to zero and clusters the remaining points around a small number of discrete value. 

This is done by using a Gaussian Mixture prior over the weights such that the most of the weights map to a gaussian with zero mean and the rest of the weights are quantized to their closest cluster centers.
From the paper:
> By fitting the mixture components alongside the weights, the weights tend to concentrate very tightly around a number of cluster components, while the cluster centers optimize themselves to give the network high predictive accuracy. Compression is achieved because we only need to encode K cluster means (in full precision) in addition to the assignment of each weight to one of these J values (using log(J) bits per weight)

## Outline of method
Following are the steps to achieve compression using the methods described in the given paper:
1. Retraining a pre trained network with gaussian mixture prior on the weights
2. Clustering the weights, merging redundant components and retrain
3. Quantize the weights by mapping them to nearest cluster mean

In [1]:
import numpy as np
import keras
from data import get_mnist

from keras.models import Model
from keras.layers import Input, Dense,  Activation, Flatten, Conv2D
from keras import optimizers

Using TensorFlow backend.
/home/archit/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Get the training data, this loads the mnist dataset if not already present
X_train, X_test, Y_train, Y_test, img_rows, img_cols, num_classes = get_mnist()

# Create a data input layer
InputLayer = Input(shape=(img_rows, img_cols,1), name="input")

# First convolution layer
conv_1 = Conv2D(25, (5, 5), strides = (2,2), activation = "relu")(InputLayer)
# Second convolution layer
conv_2 = Conv2D(50, (3, 3), strides = (2,2), activation = "relu")(conv_1)

# 2 fully connected layers with RELU activations
conv_output = Flatten()(conv_2)
fc1 = Dense(500)(conv_output)
fc1 = Activation("relu")(fc1)
fc2 = Dense(num_classes)(fc1)
PredictionLayer = Activation("softmax", name ="error_loss")(fc2)

# Fianlly, we create a model object:
model = Model(inputs=[InputLayer], outputs=[PredictionLayer])

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [3]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))

AttributeError: module 'pydot' has no attribute 'find_graphviz'

In [4]:
epochs = 1
batch_size = 256

# Adam optimizer
optimizer = optimizers.Adam(lr=0.001)

model.compile(optimizer, loss = {"error_loss": "categorical_crossentropy"}, metrics=["accuracy"])

model.fit(x=X_train, y=Y_train, 
          epochs= epochs, batch_size = batch_size,
          verbose = 1, validation_data=(X_test, Y_test))

score = model.evaluate(X_test, Y_test, verbose=0)
print('Test accuracy:', score[1])

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 15s - loss: 0.3088 - acc: 0.9146 - val_loss: 0.0998 - val_acc: 0.9699
Test accuracy: 0.9699


In [5]:
from priors import GMMPrior
from keras_helpers import fetch_weights

pi_zero = 0.099

reg_layer = GMMPrior(16, fetch_weights(model), model.get_weights(), pi_zero, name="complexity_loss")(fc2)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [6]:
model = Model(inputs=[InputLayer], outputs=[PredictionLayer, reg_layer])

In [7]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input (InputLayer)               (None, 28, 28, 1)     0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 12, 12, 25)    650                                          
____________________________________________________________________________________________________
conv2d_2 (Conv2D)                (None, 5, 5, 50)      11300                                        
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 1250)          0                                            
___________________________________________________________________________________________

In [8]:
import optimizers 
from keras_helpers import identity

tau = 0.003
N = X_train.shape[0] 

opt = optimizers.Adam(lr = [5e-4,1e-4,3e-3,3e-3],  #[unnamed, means, log(precition), log(mixing proportions)]
                      param_types_dict = ['means','gammas','rhos'])

model.compile(optimizer = opt,
              loss = {"error_loss": "categorical_crossentropy", "complexity_loss": identity},
              loss_weights = {"error_loss": 1. , "complexity_loss": tau/N},
              metrics = ['accuracy'])

In [9]:
epochs = 1
model.fit({"input": X_train,},
          {"error_loss" : Y_train, "complexity_loss": np.zeros((N,1))},
          epochs = epochs,
          batch_size = batch_size,
          verbose = 1)

Epoch 1/1
60000/60000 [==============================] - 348s - loss: 0.0692 - error_loss_loss: 0.0752 - complexity_loss_loss: -119227.7549 - error_loss_acc: 0.9778 - complexity_loss_acc: 0.0000e+00   
